**Avatars IDs**

- Mens US ASTM Sz M/40, MALE ID: "b4ec391f-4a63-41ad-8efd-f3e45c75bb28"

- Womens US ASTM Sz M/6, FEMALE ID: "ad80d4fd-cf03-4337-b42a-8df04ea07bcc"

# *Compute new positions of the garment*

In [1]:
import json
import math

#IMPORT SCENE
rute = 'store_state_transformed.json'

with open(rute) as archivo:
    data = json.load(archivo)

# Avatar & Pieces geometry data
avatar = data["avatar"]
pieces = data["pieces"]

#print("Avatar", avatar)

all_pieces_indices = []
all_pieces_position3d = []
all_pieces_position2d = []

all_sewLines = []
all_pieces_position2d = []

# Rotation
def rotate_point(x, y, theta):
    x_rotated = x * math.cos(theta) - y * math.sin(theta)
    y_rotated = x * math.sin(theta) + y * math.cos(theta)
    return x_rotated, y_rotated

for piece in data['pieces']:
    indices = piece['components']['garmentPiece']['geometry']['indices']
    position3D = piece['components']['garmentPiece']['geometry']['position3D']
    position2D = piece['components']['garmentPiece']['geometry']['position2D']    
    # Ajusta la rotación a pi/2 o -pi/2
    """
    if round(piece['components']['transform2D']['rotation'] / (math.pi / 2)) * (math.pi / 2) > 0:
        rotation = math.pi / 2
    else:
        rotation = -math.pi / 2
    """
    rotation = piece['components']['transform2D']['rotation']
    translation = piece['components']['transform2D']['translation']
    sewLine = piece['components']['garmentPiece']['sewLine']['path']    

    all_pieces_indices.extend(indices)
    all_pieces_position3d.extend(position3D)

    #GET PIECE 2D COORDINATES ROTATED & TRANSLATED
    for i in range(0, len(position2D), 2):

        x = position2D[i]
        y = position2D[i + 1]
        
        # Rotation
        x_rotated, y_rotated = rotate_point(x, y, rotation)

        # Translation
        x_translated = x_rotated + translation['x']
        y_translated = y_rotated + translation['y']
        
        # Update 2d coordinates
        position2D[i] = x_translated
        position2D[i + 1] = y_translated
    all_pieces_position2d.append({'garmentPiece_id': piece['_id'], 'position2D': position2D})

    for point in sewLine:
        x = point['position']['x']
        y = point['position']['y']
        
        # Rotar el punto
        x_rotated, y_rotated = rotate_point(x, y, rotation)
        
        # Aplicar translación
        x_translated = x_rotated + translation['x']
        y_translated = y_rotated + translation['y']
        
        # Actualizar las coordenadas del punto
        point['position']['x'] = x_translated
        point['position']['y'] = y_translated
        if 'control1' in point:
            # Rotar el punto de control1
            x_c1 = point['control1']['x']
            y_c1 = point['control1']['y']
            x_rotated_c1, y_rotated_c1 = rotate_point(x_c1, y_c1, rotation)
            
            # Aplicar translación al punto de control1 rotado
            x_translated_c1 = x_rotated_c1 + translation['x']
            y_translated_c1 = y_rotated_c1 + translation['y']
            
            # Actualizar las coordenadas del punto de control1
            point['control1']['x'] = x_translated_c1
            point['control1']['y'] = y_translated_c1

        if 'control2' in point:
            # Rotar el punto de control2
            x_c2 = point['control2']['x']
            y_c2 = point['control2']['y']
            x_rotated_c2, y_rotated_c2 = rotate_point(x_c2, y_c2, rotation)
            
            # Aplicar translación al punto de control2 rotado
            x_translated_c2 = x_rotated_c2 + translation['x']
            y_translated_c2 = y_rotated_c2 + translation['y']
            
            # Actualizar las coordenadas del punto de control2
            point['control2']['x'] = x_translated_c2
            point['control2']['y'] = y_translated_c2
    all_sewLines.append({'garmentPiece_id': piece['_id'], 'sewLine': sewLine})


transformed_poms = []

# ALL POMS TRANSFORMED POSITIONS
for pom in data["poms"]:

    garment_piece_id = pom["garmentPieceId"]

    # TRANSFORM associated with pom
    for piece in data['pieces']:
        if piece['_id'] == garment_piece_id:
            rotation = piece['components']['transform2D']['rotation']
            translation = piece['components']['transform2D']['translation']
            break

    # POM COORDS
    pom_coords = [(point["position"]["x"], point["position"]["y"]) for point in pom["position"]["_path"]]
    transformed_pom_coords = []
    for x, y in pom_coords:
        # Rotation
        x_rotated, y_rotated = rotate_point(x, y, rotation)
        # Translation
        x_translated = x_rotated + translation['x']
        y_translated = y_rotated + translation['y']
        transformed_pom_coords.append((x_translated, y_translated))
    transformed_poms.append(transformed_pom_coords)
        

print("Pieces", len(all_pieces_position3d)/3)
print("Pieces", all_pieces_position2d)


Pieces 6985.0
Pieces [{'garmentPiece_id': '66320fb05bf69c7d6fd42a2b', 'position2D': [-0.7595435950734257, 1.6568508828658655, -0.7695431221207777, 1.6569471538090597, -0.7795426640686003, 1.6570434248957089, -0.789542206016423, 1.657139695982358, -0.7995417479642456, 1.6572359670690073, -0.8095412750115976, 1.6573322380122015, -0.8195408169594203, 1.6574285090988508, -0.829540358907243, 1.6575247801855, -0.8395398859545949, 1.6576210511286942, -0.8495394279024175, 1.6577173222153434, -0.8595389549497695, 1.6578135931585376, -0.8695384968975922, 1.6579098642451868, -0.8795380313951795, 1.6580061352601085, -0.8895375658927669, 1.6581024062750302, -0.8995371078405895, 1.6581986773616795, -0.9095366348879415, 1.6582949483048737, -0.9195361768357642, 1.658391219391523, -0.9295357262338221, 1.6584874905498996, -0.939535253281174, 1.6585837614930938, -0.9495347877787614, 1.6586800325080155, -0.9595343148261134, 1.6587763034512097, -0.9695338567739361, 1.658872574537859, -0.9795333987217587, 1

In [2]:
import numpy as np

#Convert to numpy arrays & matrix
avatar_indices = np.array(avatar["indices"])
avatar_positions = np.array(avatar["positions"]).reshape(-1, 3)  # 3 colum matrix 

pieces_indices = np.array(all_pieces_indices)
pieces_positions = np.array(all_pieces_position3d).reshape(-1, 3)  # 3 colum matrix 
pieces_positions2d = np.array(all_pieces_position2d)
bezier_2d = np.array(all_sewLines)
pieces_bezierPath = np.array(all_sewLines)

print(bezier_2d)


[{'garmentPiece_id': '66320fb05bf69c7d6fd42a2b', 'sewLine': [{'position': {'x': -0.7597123196964823, 'y': 1.639325694311245}, 'type': 'LINE', '_id': '6601616da0216e4eb0a5a3b9'}, {'position': {'x': -0.7595435950734257, 'y': 1.6568508828658655}, 'type': 'SEWING', '_id': '6601616da0216e4eb0a5a3ba'}, {'position': {'x': -1.1895236753227398, 'y': 1.6609905374399567}, 'type': 'SEWING', '_id': '6601616da0216e4eb0a5a3bb'}, {'position': {'x': -1.1896923999457965, 'y': 1.6434653488853364}, 'type': 'LINE', '_id': '6601616da0216e4eb0a5a3bc'}, {'position': {'x': -1.189861124568853, 'y': 1.625940160330716}, 'type': 'SEWING', '_id': '6601616da0216e4eb0a5a3bd'}, {'position': {'x': -0.759881044319539, 'y': 1.6218005057566247}, 'type': 'SEWING', '_id': '6601616da0216e4eb0a5a3be'}]}
 {'garmentPiece_id': '66320fb05bf69c7d6fd42a3f', 'sewLine': [{'position': {'x': -0.4465853754956474, 'y': 1.4097673432856956}, 'type': 'SEWING', '_id': '6601616da0216e4eb0a5a3c9'}, {'position': {'x': -0.6005541494073767, 'y': 

In [3]:
# Compute min distances garment-avatar
min_distances = np.zeros(len(pieces_positions))
ac = np.zeros(len(pieces_positions), dtype=int) # Avatar Correspondences

# For all vertex in the garment
for i, pos_prenda in enumerate(pieces_positions):
    dist_vertex = np.linalg.norm(pos_prenda - avatar_positions, axis=1)
    # Find nearest avatar index and compute min distance
    nearest_indice_vertex = np.argmin(dist_vertex)
    min_dist = dist_vertex[nearest_indice_vertex]
    #print(distancias_vertice[indice_vertice_cercano])

    # Save min distances and avatar index correspondences
    min_distances[i] = min_dist
    ac[i] = nearest_indice_vertex



In [4]:
# Offset between garment vertex and its nearest avatar vertex
offsets = np.zeros_like(pieces_positions)

for i, indice_avatar in enumerate(ac):
    offset = np.array(pieces_positions[i])- np.array(avatar_positions[indice_avatar])  
    offsets[i]=offset

#print(offsets)



New Avatar

In [17]:
#IMPORT NEW AVATAR
avatars = 'avatars.json'

with open(avatars) as file:
    avatars_data = json.load(file)
new_avatar = avatars_data["big-male"]

new_avatar_positions= np.array(new_avatar["positions"]).reshape(-1, 3)  # 3d colum matrix


In [18]:
#Compute new pieces positions with the offset computed and its nearest avatar index from the new avatar (Avatars in seddi have the same mesh)
new_pieces_positions = np.zeros_like(pieces_positions)

for i, indice_avatar in enumerate(ac):
    new_pieces_positions[i] = np.array(new_avatar_positions[indice_avatar]) + np.array(offsets[i])

print(new_pieces_positions)


[[-0.05909815  1.53366232 -0.02197736]
 [-0.05101374  1.5339607  -0.03589756]
 [-0.04223864  1.53225863 -0.04688915]
 ...
 [-0.33529577  1.37586892 -0.12225011]
 [-0.33530456  1.28082085  0.00230249]
 [-0.27652331  1.4203496   0.03836989]]


In [19]:
#Export SCENE TO JSON for AUTHOR

import json
#IMPORT SCENE
rute = 'store_state_transformed.json'

with open(rute) as archivo:
    data = json.load(archivo)


pieces = data["pieces"]
new_list_positions = new_pieces_positions.reshape(-1).tolist()

for piece in data['pieces']:
    indices = piece['components']['garmentPiece']['geometry']['indices']
    positions = piece['components']['garmentPiece']['geometry']['position3D']

    # Len atucal piece
    length_piece = len(positions)
    # Get new positions actual piece
    new_positions = new_list_positions[:length_piece]

    # Remplace for new positions
    piece['components']['garmentPiece']['geometry']['position3D'] = new_positions
    
    # Update new_pieces_positions to next piece
    new_list_positions = new_list_positions[length_piece:]

with open("output.json", "w") as outfile: 
      json.dump(data, outfile)
print("Datos exportados a 'output.json'")

Datos exportados a 'output.json'


DEFORMATION METRICS FOR 2D PATTERN

In [ ]:
#Vertex defined by POM POINTS
import matplotlib.pyplot as plt


#GET POMS COORDINATES
class Coordinates:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

poms = data["poms"]
#print(transformed_poms[0][0][0])
pom_C0=Coordinates(transformed_poms[0][0][0],transformed_poms[0][0][1])
pom_C1=Coordinates(transformed_poms[0][1][0],transformed_poms[0][1][1])


POM = transformed_poms
print("POM:","C0:",pom_C0.x,pom_C0.y,"C1:",pom_C1.x,pom_C1.y)

#print(pieces_positions2d)
pieces_2d = []
for piece_data in pieces_positions2d:
    position2d = piece_data['position2D']
    for i in range(0, len(position2d), 2):
        x = position2d[i]
        y = position2d[i + 1]
        pieces_2d.append(Coordinates(x, y))


# PLOT
plt.figure(figsize=(8, 6))

plt.plot([pom_C0.x, pom_C1.x], [pom_C0.y, pom_C1.y], marker='o', label='POM')


plt.scatter([position.x for position in pieces_2d], 
            [position.y for position in pieces_2d], 
            color='red',s=1, label='Vertex 2d Pieces')


plt.xlabel('Coordenada x')
plt.ylabel('Coordenada y')
plt.title('PATTERN 2D')
plt.legend()

plt.grid(True)
plt.show()





In [ ]:
#COMPUTE DEFORMATION METRIC 3D: EDGE LENGTH

def compute_edge_lengths(points, indices):
    edge_lengths = []
    for i in range(0, len(indices), 3):
        # Obtener los índices de los puntos que forman el triángulo
        p1_index, p2_index, p3_index = indices[i], indices[i+1], indices[i+2]

        # Obtener las posiciones 3D de los puntos del triángulo
        p1 = points[p1_index]
        p2 = points[p2_index]
        p3 = points[p3_index]

        # Calcular la longitud de cada arista usando np.linalg.norm
        edge_ab = np.linalg.norm(p2 - p1)
        edge_bc = np.linalg.norm(p3 - p2)
        edge_ca = np.linalg.norm(p1 - p3)

        edge_lengths.append(edge_ab)
        edge_lengths.append(edge_bc)
        edge_lengths.append(edge_ca)

    return edge_lengths

def compute_edge_difference(avatar1_lengths, avatar2_lengths):
    if len(avatar1_lengths) != len(avatar2_lengths):
        raise ValueError("AVATARS NEED SAME NUMBER OF EDGES.")
    
    differences = np.abs(np.array(avatar1_lengths) - np.array(avatar2_lengths))
    return differences
    
def compute_edge_strain(current_length, original_length):
    if original_length == 0:
        return 0
    return current_length / original_length - 1

def compute_total_strain(current_lengths, original_lengths):
    total_strain = []
    for current_length, original_length in zip(current_lengths, original_lengths):
        strain = compute_edge_strain(current_length, original_length)
        total_strain.append(strain)
    
    return np.array(total_strain)


avatar1_3dpoints = pieces_positions
avatar2_3dpoints = new_pieces_positions

# COMPUTE EDGE LENGTHS Avatar 1 & Avatar 2
avatar1_edge_lengths = compute_edge_lengths(avatar1_3dpoints,pieces_indices)
avatar2_edge_lengths = compute_edge_lengths(avatar2_3dpoints,pieces_indices)

# Compute the difference in edge lengths between the two avatars
diff_edges = compute_edge_difference(avatar1_edge_lengths, avatar2_edge_lengths)

target_strain = compute_total_strain(np.array(avatar2_edge_lengths),np.array(avatar1_edge_lengths)).flatten()

print("Diferencia en longitudes de aristas entre los dos avatares:")

# Crear el histograma
plt.hist(target_strain, bins=50, color='skyblue', edgecolor='black')

# Añadir etiquetas y título
plt.xlabel('Diferencia')
plt.ylabel('Frecuencia')
plt.title('STRAIN Histograma de % diferencias entre aristas de triángulos')

# Mostrar el histograma
plt.show()

print(len(target_strain))

In [ ]:
#1. HOW POM AFFECT BEZIER CONTROL POINTS: SEWLINE

# x : Slider value
#f(x) : return  NEW POSITIONS POINTS WITH DISPLACEMENT

# DIBUJAR
positions_array = np.array([(point['position']['x'], point['position']['y']) for piece in bezier_2d for point in piece['sewLine']])
plt.plot([pom_C0.x, pom_C1.x], [pom_C0.y, pom_C1.y], marker='o', label='POM')

# Graficar los puntos
plt.scatter(positions_array[:, 0], positions_array[:, 1], color='green', label='Puntos de control')

# Mostrar la gráfica
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.title('Puntos de control de la curva de Bezier')
plt.legend()
plt.grid(True)
plt.show()


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

def plot_bezier_curves(bezier_2d):
    # Función para calcular el punto en la curva de Bézier
    def bezier(t, P0, P1=None, P2=None, P3=None):
        if P1 is None or P2 is None:
            # Si solo hay dos puntos, calcula la interpolación lineal
            return [(1-t)*P0[i] + t*P3[i] for i in range(2)]
        else:
            # Si hay cuatro puntos, calcula la curva de Bézier
            return [(1-t)**3 * P0[i] + 3 * (1-t)**2 * t * P1[i] + 3 * (1-t) * t**2 * P2[i] + t**3 * P3[i] for i in range(2)]

    # Crear figura y ejes una vez
    plt.figure()
    ax = plt.gca()
    all_curves_points = []

    # Iterar sobre cada elemento en bezier_2d
    for bezier_data in bezier_2d:
        sewLine = bezier_data['sewLine']
        
        # Generar valores de t (parámetro)
        t_values = np.linspace(0, 1, 2000)

        # Iterar sobre cada par de puntos en sewLine
        for i in range(len(sewLine) - 1):
            P0 = [sewLine[i]['position']['x'], sewLine[i]['position']['y']]
            P3 = [sewLine[i+1]['position']['x'], sewLine[i+1]['position']['y']]
            
            if 'control1' in sewLine[i] and 'control2' in sewLine[i]:
                P1 = [sewLine[i]['control1']['x'], sewLine[i]['control1']['y']]
                P2 = [sewLine[i]['control2']['x'], sewLine[i]['control2']['y']]
            else:
                P1 = None
                P2 = None
            
            curve_points = np.array([bezier(t, P0, P1, P2, P3) for t in t_values])
            all_curves_points.append(curve_points)

            ax.plot(curve_points[:,0], curve_points[:,1], label=f'Curva {i+1} de Bézier')

        # Unir el último punto con el primero
        P0_last = [sewLine[0]['position']['x'], sewLine[0]['position']['y']]
        P3_first = [sewLine[-1]['position']['x'], sewLine[-1]['position']['y']]

        if 'control1' in sewLine[-1] and 'control2' in sewLine[-1]:
            P2_first = [sewLine[-1]['control1']['x'], sewLine[-1]['control1']['y']]
            P1_first = [sewLine[-1]['control2']['x'], sewLine[-1]['control2']['y']]
        else:
            P1_first = None
            P2_first = None

        # Verificamos si hay puntos de control y los incluimos en el cálculo
        if P1_first is not None and P2_first is not None:
            curve_points = np.array([bezier(t, P0_last, P1_first, P2_first, P3_first) for t in t_values])
        else:
            curve_points = np.array([bezier(t, P0_last, None, None, P3_first) for t in t_values])
        all_curves_points.append(curve_points)

        ax.plot(curve_points[:,0], curve_points[:,1], label=f'Curva cerrada de Bézier')
        # Añadir puntos de control y primeros y últimos puntos
        for i, point in enumerate(sewLine):
            ax.plot(point['position']['x'], point['position']['y'], 'ro', label=f'Punto {i}')
            if 'control1' in point:
                ax.plot(point['control1']['x'], point['control1']['y'], 'go', label=f'Control 1 {i}')
            if 'control2' in point:
                ax.plot(point['control2']['x'], point['control2']['y'], 'bo', label=f'Control 2 {i}')

    # Personalizar la leyenda
    #ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

    # Configurar ejes y etiquetas
    ax.set_xlabel('Coordenada X')
    ax.set_ylabel('Coordenada Y')
    ax.set_title('Curvas de Bézier')
    ax.grid(True)

    # Mostrar la gráfica
    plt.show()
    return all_curves_points

# Llamar a la función para graficar las curvas de Bézier
all_curve_points = plot_bezier_curves(bezier_2d)





#print(len(points_inside_array))


In [ ]:
boundary_curves= np.concatenate(all_curve_points, axis=0)

points_inside = []

# Umbral de distancia
threshold = 0.001  # Puedes ajustar este valor según tus necesidades

# Iterar sobre los objetos Coordinates en pieces_2d
for coord in pieces_2d:
    # Coordenadas x e y del objeto Coordinates
    x = coord.x
    y = coord.y
    
    # Calcular la distancia euclidiana entre el punto (x, y) y cada punto en boundary_curves
    distances = np.linalg.norm(boundary_curves - np.array([x, y]), axis=1)
    
    # Verificar si alguna de las distancias es menor que el umbral
    if any(distances < threshold):
        points_inside.append((x, y))

# Convertir points_inside a un array NumPy para facilitar la manipulación
points_inside_array = np.array(points_inside)
# Extraer las coordenadas x e y de los puntos dentro de points_inside_array
x_inside = points_inside_array[:, 0]
y_inside = points_inside_array[:, 1]

# Dibujar los puntos
plt.scatter(x_inside, y_inside, color='blue', label='Boundary 2d Vertex',s=1)
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.title('Boundary')
plt.legend()
plt.show()


In [ ]:
import numpy as np

# Función para calcular los puntos de la curva de Bezier
def bezier(t, P0, P1, P2, P3):
    return (1 - t)**3 * P0 + 3 * (1 - t)**2 * t * P1 + 3 * (1 - t) * t**2 * P2 + t**3 * P3

# Función para encontrar el segmento y el parámetro t de un punto en las curvas de Bézier
def find_segment_and_t(all_curve_points, point):
    """
    Encuentra el segmento y el parámetro t de un punto en las curvas de Bézier.

    Args:
    all_curve_points (list): Lista de puntos de las curvas de Bézier.
    point (list): Coordenadas [x, y] del punto a buscar.

    Returns:
    tuple: El índice del segmento y el valor de t correspondiente.
    """
    # Inicializar las variables para el punto más cercano y su distancia mínima
    closest_point = None
    min_distance = float('inf')
    segment_index = -1
    
    # Iterar sobre cada conjunto de puntos de las curvas de Bézier
    for index, curve_points in enumerate(all_curve_points):
        # Calcular la distancia euclidiana entre el punto dado y cada punto en la curva de Bézier
        distances = np.linalg.norm(curve_points - point, axis=1)
        
        # Encontrar el punto más cercano en la curva de Bézier
        closest_index = np.argmin(distances)
        closest_distance = distances[closest_index]
        
        # Si la distancia al punto más cercano es menor que la mínima registrada hasta ahora, actualizar
        if closest_distance < min_distance:
            closest_point = curve_points[closest_index]
            min_distance = closest_distance
            segment_index = index
    
    # Calcular el valor de t correspondiente al punto más cercano
    if closest_point is not None:
        t_value = np.where(np.all(all_curve_points[segment_index] == closest_point, axis=1))[0][0] / (len(all_curve_points[segment_index]) - 1)
        return segment_index, t_value
    else:
        return None, None


# Lista para almacenar los segmentos y los parámetros t para todos los puntos del borde
boundary_vertex_segment_t = []

# Iterar sobre todos los puntos del borde
for point_to_find in points_inside_array:
    # Llamar a la función para encontrar el segmento y el parámetro t del punto
    segment_index, t_value = find_segment_and_t(all_curve_points, point_to_find)
    
    # Si se encuentra el punto en alguna curva de Bézier, almacenar el resultado
    if segment_index is not None:
        boundary_vertex_segment_t.append((point_to_find, segment_index, t_value))

# Mostrar los resultados
for idx, (point_to_find, segment_index, t_value) in enumerate(boundary_vertex_segment_t):
    print(f"Punto {idx}: {point_to_find} está en el segmento {segment_index} con un valor de t = {t_value}.")
print(boundary_vertex_segment_t)

In [ ]:
print(points_inside_array)
print(bezier_2d)

# Definir puntos del borde de la malla en funcion de la sewLine

# Función para recalcular un punto en la curva de Bezier dado un segmento y un valor de t
def recalculate_point(all_curve_points, segment_index, t_value):

    # Obtener los puntos del segmento dado
    curve_points = all_curve_points[segment_index]
    
    # Calcular el índice correspondiente al valor de t
    index = int(t_value * (len(curve_points) - 1))
    
    # Devolver las coordenadas del punto en ese índice
    return curve_points[index]

recalculated_points = []
for point_to_find in boundary_vertex_segment_t:
    # Llamar a la función para encontrar el segmento y el parámetro t del punto
    segment_index=point_to_find[1]
    t_value=point_to_find[2]
    recalculated_point = recalculate_point(all_curve_points, segment_index, t_value)

    # Si se encuentra el punto en alguna curva de Bézier, almacenar el resultado
    if segment_index is not None:
        recalculated_points.append(recalculated_point)

# Recalcular el punto usando el segmento y el valor de t

print(f"El punto recalculado es: {recalculated_points}")
print(len(recalculated_points))
# Pintar los puntos recalculados
for point in recalculated_points:
    plt.scatter(point[0], point[1], color='blue', label='Punto Recalculado',s=1)


    # Configuraciones adicionales del gráfico
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.title('Punto Recalculado en Curva de Bézier')
plt.grid(True)
    
    # Mostrar el gráfico
plt.show()


In [ ]:
import numpy as np
bezier_curve = np.concatenate(all_curve_points, axis=0)

def find_intersection_points(bezier_curve, ray, orientation):
    intersection_points = []
    if(orientation=='horizontal'):

        for i in range(len(bezier_curve) - 1):
            # Check for intersection between line segment and ray
            y1, y2 = bezier_curve[i][1], bezier_curve[i+1][1]
            if (y1 >= ray and y2 <= ray) or (y1 <= ray and y2 >= ray):
                # Interpolate x value at intersection point
                t = (ray - y1) / (y2 - y1)
                intersection_x = bezier_curve[i][0] + t * (bezier_curve[i+1][0] - bezier_curve[i][0])
                intersection_points.append((intersection_x, ray))
        return intersection_points
    else:     
        for i in range(len(bezier_curve) - 1):
        # Obtener coordenadas x de los puntos del segmento
            x1, x2 = bezier_curve[i][0], bezier_curve[i+1][0]
            
            # Verificar si el segmento intersecta con la línea vertical
            if (x1 >= ray and x2 <= ray) or (x1 <= ray and x2 >= ray):
                # Interpolar el valor y en el punto de intersección
                t = (ray - x1) / (x2 - x1)
                intersection_y = bezier_curve[i][1] + t * (bezier_curve[i+1][1] - bezier_curve[i][1])
                intersection_points.append((ray, intersection_y))
        return intersection_points

def closest_points_to_P(intersection_points, P, orientation):
    if(orientation=='horizontal'):
        left_point, right_point = None, None
        min_dist_left, min_dist_right = float('inf'), float('inf')
        for point in intersection_points:
            dist = np.linalg.norm(np.array(P) - np.array(point))
            if point[0] < P[0] and dist < min_dist_left:
                left_point = point
                min_dist_left = dist
            elif point[0] > P[0] and dist < min_dist_right:
                right_point = point
                min_dist_right = dist

        return left_point, right_point
    else:
        above_point, below_point = None, None
        min_dist_above, min_dist_below = float('inf'), float('inf')
        for point in intersection_points:
            dist = np.linalg.norm(np.array(P) - np.array(point))
            # Verificar si el punto está arriba o abajo de P en el eje y
            if point[1] > P[1] and dist < min_dist_above:
                above_point = point
                min_dist_above = dist
            elif point[1] < P[1] and dist < min_dist_below:
                below_point = point
                min_dist_below = dist
        return above_point, below_point


def nearest_indices(array_nearest, array_points):
    indices = []
    for point in array_nearest:
        # Calcular las distancias entre el punto actual y todos los puntos en array_points
        distances = np.linalg.norm(array_points - point, axis=1)
        # Encontrar el índice del punto más cercano
        nearest_index = np.argmin(distances)
        indices.append(nearest_index)
    return indices


def boundary_points(intersection_points, point_left):
    # Calcular las distancias euclidianas entre los puntos de intersección y point_left
    distances = np.linalg.norm(intersection_points - np.array([point_left[0], point_left[1]]), axis=1)
    
    # Obtener los índices de los puntos más cercanos (ordenados de menor a mayor distancia)
    nearest_indices = np.argsort(distances)
    
    # Tomar los dos puntos más cercanos
    nearest_boundary_points = [intersection_points[i] for i in nearest_indices[:2]]
    
    return nearest_boundary_points


def calculate_angle(point1, point2):
    """
    Calcula el ángulo entre dos puntos y el eje x.
    """
    dx = point2[0] - point1[0]
    dy = point2[1] - point1[1]
    return np.arctan2(dy, dx)

def reorder_indices(reference_points, P):
    """
    Reordena los índices de los puntos de referencia en función del ángulo con el punto P.
    """
    angles = [calculate_angle(P, point) for point in reference_points]
    indices = np.argsort(angles)
    return indices




def is_convex_quadrilateral(points):
    """
    Verifica si los puntos forman un cuadrilátero convexo.
    """
    def cross_product(p1, p2, p3):
        """
        Calcula el producto cruzado de dos vectores formados por los puntos p1, p2, y p3.
        """
        return (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0])

    n = len(points)
    for i in range(n):
        # Calcula el producto cruzado para cada tripleta de puntos adyacentes
        j = (i + 1) % n
        k = (i + 2) % n
        cross = cross_product(points[i], points[j], points[k])
        # Si el producto cruzado es negativo, indica que los puntos no están en orden convexo
        if cross < 0:
            return False
    return True


def calculate_weights(points, P):
    x_p, y_p = P
    weights = []
    for x, y in points:
        distance = np.sqrt((x - x_p)**2 + (y - y_p)**2)
        weights.append(1 / distance)  # Inverse distance weighting
    total_weight = sum(weights)
    return [w / total_weight for w in weights]  # Normalize weights
    """
    A = points[0]
    B = points[1]
    C = points[2]
    v0 = C - A
    v1 = B - A
    v2 = P - A

    # Calculate dot products
    dot00 = np.dot(v0, v0)
    dot01 = np.dot(v0, v1)
    dot02 = np.dot(v0, v2)
    dot11 = np.dot(v1, v1)
    dot12 = np.dot(v1, v2)

    # Compute barycentric coordinates
    inv_denom = 1 / (dot00 * dot11 - dot01 * dot01)
    u = (dot11 * dot02 - dot01 * dot12) * inv_denom
    v = (dot00 * dot12 - dot01 * dot02) * inv_denom
    w = 1 - u - v
    return u, v, w
        """

def bilinear_interpolation(points, weights):
    interpolated_x = sum(w * x for w, (x, _) in zip(weights, points))
    interpolated_y = sum(w * y for w, (_, y) in zip(weights, points))
    """
    u=weights[0]
    v=weights[1]
    w=weights[2]

    weighted_sum = u * points[0] + v * points[1] + w * points[2]

    
    return weighted_sum[0], weighted_sum[1]
    """
    return interpolated_x,interpolated_y


def vertex_weights_nearest_points(P, orientation, bezier_curve, points_inside_array):
    #plt.figure(figsize=(8, 6))

    if orientation == 'horizontal':
        ray = P[1]
        #plt.axhline(y=ray, color='gray', linestyle='--', label='Ray')

    else:
        ray = P[0]
        #plt.axvline(x=ray, color='gray', linestyle='--', label='Ray')

    #plt.scatter(*zip(*points_inside_array), color='blue',s=1, label='Points Inside')
    #plt.scatter(*zip(P), color='green', label='P')

    intersection_points = find_intersection_points(bezier_curve, ray, orientation)

    #plt.scatter(*zip(*intersection_points), color='red',s=5, label='Intersection Points')
    #plt.plot(*zip(*bezier_curve), color='black', label='Bezier Curve')

    left_point, right_point = closest_points_to_P(intersection_points, P, orientation)

    #plt.scatter(*zip(left_point), color='orange', label='Left Point')
    #plt.scatter(*zip(right_point), color='purple', label='Right Point')

    nearest_boundary_points_left = boundary_points(points_inside_array, left_point)
    nearest_boundary_points_right = boundary_points(points_inside_array, right_point)
    nearest_boundary_points = nearest_boundary_points_left + nearest_boundary_points_right
    #plt.scatter(*zip(*nearest_boundary_points_left), color='cyan', label='Nearest Boundary Points (Left)')
    #plt.scatter(*zip(*nearest_boundary_points_right), color='magenta', label='Nearest Boundary Points (Right)')

    nearest_indices_of_points = nearest_indices(nearest_boundary_points, points_inside_array)

    reordered_indices = reorder_indices(points_inside_array[nearest_indices_of_points], P)

    reordered_nearest_indices_of_points = [nearest_indices_of_points[i] for i in reordered_indices]


    weights = calculate_weights(points_inside_array[reordered_nearest_indices_of_points], P)
    #plt.xlabel('X')
    #plt.ylabel('Y')
    #plt.title('Intersection Points and Nearest Boundary Points')
    #plt.legend()
    #plt.grid(True)
    #plt.show()
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(*zip(*intersection_points), color='red',s=5, label='Intersection Points')
    plt.scatter(*zip(P), color='green', label='P')
    plt.scatter(*zip(*nearest_boundary_points_left), color='cyan', label='Nearest Boundary Points (Left)')
    plt.scatter(*zip(*nearest_boundary_points_right), color='magenta', label='Nearest Boundary Points (Right)')
    plt.scatter(*zip(left_point), color='orange', label='Left Point')
    plt.scatter(*zip(right_point), color='purple', label='Right Point')

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Intersection Points and Nearest Boundary Points')
    plt.legend()
    plt.grid(True)
    plt.show()
    """
    return weights, reordered_nearest_indices_of_points

# Example usage:
#points_array = [(-0.35, 1.35),(-0.20043471708731297, 1.20109981477757)]   # Point on ray


points_array = []

for piece_data in pieces_positions2d:
    position2d = piece_data['position2D']
    for i in range(0, len(position2d), 2):
        x = position2d[i]
        y = position2d[i + 1]
        point = [x, y]
        if not np.any(np.all(points_inside_array == point, axis=1)):
            points_array.append(point)

weights_array = []
nearest_boundary_vertex_array = []
P_recalculated_array = []
orientation='vertical'
# Iterate over each point
for P in points_array:
    # Compute weights and nearest boundary vertices
    weights, nearest_boundary_vertex = vertex_weights_nearest_points(P, orientation, bezier_curve, points_inside_array)
    #print(points_inside_array[nearest_boundary_vertex])
    # Perform bilinear interpolation
    P_recalculated = bilinear_interpolation(points_inside_array[nearest_boundary_vertex], weights)
    
    # Append results to arrays
    weights_array.append(weights)
    nearest_boundary_vertex_array.append(nearest_boundary_vertex)
    P_recalculated_array.append(P_recalculated)
    #print("Weights:", weights)

    #print("Original Point P:", (P[0], P[1]),"Interpolated position at point P:", (P_recalculated[0], P_recalculated[1]))
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(P[0], P[1], color='blue', label='Puntos originales')
    plt.scatter(P_recalculated[0], P_recalculated[1], color='blue', label='Puntos recalculated')
    plt.plot(*zip(*bezier_curve), color='black', label='Bezier Curve')

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Intersection Points and Nearest Boundary Points')
    plt.legend()
    plt.grid(True)
    plt.show()
    """

#P_recalculated_array = recalculate_vertex_points(points_inside_array,nearest_boundary_vertex_array,weights_array)


# Plotting
# Plot P, left_point, right_point

# Plot nearest boundary points




print(nearest_boundary_vertex_array)

In [ ]:
import math
import copy


def get_sewline_by_id(id,bezier):
    for piece in bezier:
        if piece['garmentPiece_id'] == id:
            sewLine = piece['sewLine']
            return sewLine
    return None

def update_sewline_in_pieces(sewline, id,bezier):
    # Hacer una copia de bezier_2d
    for piece in bezier:
        if piece['garmentPiece_id'] == id:
            piece['sewLine'] = sewline



def calcular_desplazamiento(p1,p2, orientacion, diferencia_longitud):
    ajuste_mitad =  diferencia_longitud / 2

    if orientacion == 1:
        ajuste_mitad = diferencia_longitud

    desplazamiento_x = ajuste_mitad
    desplazamiento_y = ajuste_mitad

    if orientacion == 0:

        desplazamiento_y = 0
    elif orientacion == 1:
        desplazamiento_x = 0
        
    elif orientacion == 2:
      dx = p2.x - p1.x
      dy = p2.y - p1.y
      dist = math.sqrt(dx * dx + dy * dy)

      ux = math.abs(dx / dist)
      uy = dy / dist


      direccionOriginal = { x: ux, y: uy }

      desplazamiento_x = direccionOriginal.x * ajuste_mitad
      desplazamiento_y= direccionOriginal.y * ajuste_mitad

    return desplazamiento_x, desplazamiento_y



def desplazar_puntos_sewline_en_pom(sewline, indices_desplazar, despX, despY):
    for idx in indices_desplazar:
        if 0 <= idx < len(sewline):
            point = sewline[idx]
            #print(point)
            
            # Desplazar el punto principal
            point['position']['x'] -= despX
            point['position']['y'] -= despY

            # Si el punto tiene puntos de control, también desplazarlos
            #if 'control1' in point:
            #    point['control1']['x'] -= despX
            #    point['control1']['y'] -= despY
            #if 'control2' in point:
            #    point['control2']['x'] -= despX
            #    point['control2']['y'] -= despY






slider_Value= np.sqrt((pom_C1.x - pom_C0.x)**2 + (pom_C1.y - pom_C0.y)**2) #0.57 -> valor Slider 57cm
newSliderValue = 0.9   #Ahora quiero que sea 60cm
diferencia_longitud = newSliderValue - slider_Value
bezier_2d_copy = copy.deepcopy(bezier_2d)




recalculated_boundary_points = []
recalculated_vertex_points = []


original_bezier = copy.deepcopy(pieces_bezierPath)
print(original_bezier)

bezier_Path = copy.deepcopy(pieces_bezierPath)

def f(dif,bezierPath):
    despX, despY = calcular_desplazamiento(pom_C0,pom_C1,poms[0]['orientation'],dif)
    # Desplazar los puntos de la sewline del POM
    if orientation!='vertical':
        sewLine_POM_0 = get_sewline_by_id(poms[0]['POM'][0][0]['piece'],bezierPath)
        desplazar_puntos_sewline_en_pom(sewLine_POM_0, poms[0]['POM'][0][0]['nodes']['sewLine'],despX,despY)
        update_sewline_in_pieces(sewLine_POM_0, poms[0]['POM'][0][0]['piece'],bezierPath)

        # Desplazar los puntos de la sewline del POM
        sewLine_POM_1 = get_sewline_by_id(poms[0]['POM'][1][0]['piece'],bezierPath)
        desplazar_puntos_sewline_en_pom(sewLine_POM_1, poms[0]['POM'][1][0]['nodes']['sewLine'],-despX,-despY)
        update_sewline_in_pieces(sewLine_POM_1, poms[0]['POM'][1][0]['piece'],bezierPath)
    else:
        sewLine_POM_1 = get_sewline_by_id(poms[0]['POM'][1][0]['piece'],bezierPath)
        desplazar_puntos_sewline_en_pom(sewLine_POM_1, poms[0]['POM'][1][0]['nodes']['sewLine'],despX,despY)
        update_sewline_in_pieces(sewLine_POM_1, poms[0]['POM'][1][0]['piece'],bezierPath)
        
    all_curve_points=plot_bezier_curves(bezierPath)
    for point_to_find in boundary_vertex_segment_t:
        # Llamar a la función para encontrar el segmento y el parámetro t del punto
        segment_index=point_to_find[1]
        t_value=point_to_find[2]
        recalculated_point = recalculate_point(all_curve_points, segment_index, t_value)

        # Si se encuentra el punto en alguna curva de Bézier, almacenar el resultado
        if segment_index is not None:
            recalculated_boundary_points.append(recalculated_point)




    return bezierPath
    #return new_2d_vertex_positions


new_bezier_2d_positions = f(diferencia_longitud,bezier_Path)


weights_array = []
nearest_boundary_vertex_array = []
recalculated_vertex_points = []
orientation='vertical'

for P in points_array:
    # Compute weights and nearest boundary vertices
    weights, nearest_boundary_vertex = vertex_weights_nearest_points(P, orientation, bezier_curve, points_inside_array)
    # Append results to arrays
    weights_array.append(weights)
    nearest_boundary_vertex_array.append(nearest_boundary_vertex)


for i, P in enumerate(points_array):
    recalculated_point_index = nearest_boundary_vertex_array[i]
    recalculated = bilinear_interpolation(np.array(recalculated_boundary_points)[recalculated_point_index], weights_array[i])
    recalculated_vertex_points.append(recalculated)



recalculated_vertex_points_x = [point[0] for point in recalculated_vertex_points]
recalculated_vertex_points_y = [point[1] for point in recalculated_vertex_points]

# Dibuja los puntos originales y recalculados
plt.scatter(recalculated_vertex_points_x, recalculated_vertex_points_y, color='orange',s=1, label='Puntos recalculados')



#DIBUJAR
old_positions_array = np.array([(point['position']['x'], point['position']['y']) for piece in bezier_2d for point in piece['sewLine']])
old_control_points_array = np.array([(point['control1']['x'], point['control1']['y']) for piece in bezier_2d for point in piece['sewLine'] if 'control1' in point] +
                                [(point['control2']['x'], point['control2']['y']) for piece in bezier_2d for point in piece['sewLine'] if 'control2' in point])
plt.scatter(old_control_points_array[:, 0], old_control_points_array[:, 1], color='black', label='Old Puntos de control', marker='o', facecolors='none')

# Graficar los puntos
plt.scatter(old_positions_array[:, 0], old_positions_array[:, 1], color='black', label='Old Puntos de control')

new_positions_array = np.array([(point['position']['x'], point['position']['y']) for piece in new_bezier_2d_positions for point in piece['sewLine']])
plt.plot([pom_C0.x, pom_C1.x], [pom_C0.y, pom_C1.y], marker='o', label='POM')
# Obtener los puntos de control de bezier_2d
new_control_points_array = np.array([(point['control1']['x'], point['control1']['y']) for piece in new_bezier_2d_positions for point in piece['sewLine'] if 'control1' in point] +
                                [(point['control2']['x'], point['control2']['y']) for piece in new_bezier_2d_positions for point in piece['sewLine'] if 'control2' in point])

for point in recalculated_boundary_points:
    plt.scatter(point[0], point[1], color='blue',s=1)

# Pintar los puntos de control
plt.scatter(new_control_points_array[:, 0], new_control_points_array[:, 1], color='green', label='New Puntos de control', marker='o', facecolors='none')
plt.scatter(new_positions_array[:, 0], new_positions_array[:, 1], color='green', label='New Puntos de control')

plt.scatter([position.x for position in pieces_2d], 
            [position.y for position in pieces_2d], 
            color='red',s=1, label='Vertex 2d Pieces')
# Mostrar la gráfica
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.title('Puntos de control de la curva de Bezier')
plt.legend()
plt.grid(True)
plt.show()




In [ ]:

print(len(recalculated_vertex_points))

for i, P in enumerate(points_array):
    recalculated_point_index = nearest_boundary_vertex_array[i]
    recalculated = bilinear_interpolation(np.array(recalculated_boundary_points)[recalculated_point_index], weights_array[i])
    recalculated_vertex_points.append(recalculated)




recalculated_vertex_points_x = [point[0] for point in recalculated_vertex_points]
recalculated_vertex_points_y = [point[1] for point in recalculated_vertex_points]

# Dibuja los puntos originales y recalculados
plt.scatter(recalculated_vertex_points_x, recalculated_vertex_points_y, color='orange',s=1, label='Puntos recalculados')
plt.show()


In [ ]:
# CALCULATE  INIT PARAMETERS TENSORS: POM, BEZIER, BOUNDARY VERTEX, INTERIOR VERTEX 


#GET BOUNDARY: boundary_vertex_segment_t_tensor &  INTERION: weights_array_tensor and nearest_boundary_vertex_array_tensor

import torch

# Inicializar una lista vacía para almacenar los tensores individuales
boundary_vertex_segment_t_tensor = []

# Iterar sobre todos los puntos del borde
for point_to_find in points_inside_array:
    # Llamar a la función para encontrar el segmento y el parámetro t del punto
    segment_index, t_value = find_segment_and_t(all_curve_points, point_to_find)
    
    # Si se encuentra el punto en alguna curva de Bézier, almacenar el resultado
    if segment_index is not None:
        # Crear un tensor con los resultados actuales
        result_tensor = torch.tensor([[point_to_find[0], point_to_find[1], segment_index, t_value]], dtype=torch.float32)
        
        # Agregar el tensor de resultados a la lista
        boundary_vertex_segment_t_tensor.append(result_tensor)

# Concatenar todos los tensores individuales en un solo tensor
if boundary_vertex_segment_t_tensor:
    boundary_vertex_segment_t_tensor = torch.cat(boundary_vertex_segment_t_tensor, dim=0)

# Imprimir el tensor resultante
print("boundary_vertex_segment_t_tensor:",boundary_vertex_segment_t_tensor)

weights_array_tensor = []
nearest_boundary_vertex_array_tensor = []
recalculated_vertex_points_tensor = []
orientation = 'horizontal'


for P in points_array:
    # Calcular pesos y puntos de borde más cercanos
    weights, nearest_boundary_vertex = vertex_weights_nearest_points(P, orientation, bezier_curve, points_inside_array)
    
    # Convertir las listas de pesos y puntos de borde más cercanos en tensores
    weights_tensor = torch.tensor(weights, dtype=torch.float32)
    nearest_boundary_vertex_tensor = torch.tensor(nearest_boundary_vertex, dtype=torch.float32)
    
    # Añadir los tensores resultantes a las listas de tensores
    weights_array_tensor.append(weights_tensor)
    nearest_boundary_vertex_array_tensor.append(nearest_boundary_vertex_tensor)

# Convertir las listas de tensores a un solo tensor
if weights_array_tensor:
    weights_array_tensor = torch.stack(weights_array_tensor, dim=0)
if nearest_boundary_vertex_array_tensor:
    nearest_boundary_vertex_array_tensor = torch.stack(nearest_boundary_vertex_array_tensor, dim=0)

# Imprimir los tensores resultantes
print("weights_array_tensor", weights_array_tensor)
print("nearest_boundary_vertex_array_tensor:",nearest_boundary_vertex_array_tensor)




In [ ]:


def bezier_curves_tensor(bezier_tensor):

    def bezier(t, P0, P1=None, P2=None, P3=None):
        if P1 is None or P2 is None:
            # Si solo hay dos puntos, calcula la interpolación lineal
            return torch.tensor([(1-t)*P0[i] + t*P3[i] for i in range(2)])
        else:
            # Si hay cuatro puntos, calcula la curva de Bézier
            return torch.tensor([(1-t)**3 * P0[i] + 3 * (1-t)**2 * t * P1[i] + 3 * (1-t) * t**2 * P2[i] + t**3 * P3[i] for i in range(2)])

    all_curves_points_t = []

    # Iterar sobre cada elemento en bezier_2d
    for bezier_data in bezier_tensor:
        sewLine = bezier_data['sewLine']
        
        # Generar valores de t (parámetro)
        t_values = torch.linspace(0, 1, 2000)

        # Iterar sobre cada par de puntos en sewLine
        for i in range(len(sewLine) - 1):
            P0 = torch.tensor([sewLine[i]['position']['x'], sewLine[i]['position']['y']])
            P3 = torch.tensor([sewLine[i+1]['position']['x'], sewLine[i+1]['position']['y']])
            
            if 'control1' in sewLine[i] and 'control2' in sewLine[i]:
                P1 = torch.tensor([sewLine[i]['control1']['x'], sewLine[i]['control1']['y']])
                P2 = torch.tensor([sewLine[i]['control2']['x'], sewLine[i]['control2']['y']])
            else:
                P1 = None
                P2 = None
            
            curve_points = torch.stack([bezier(t, P0, P1, P2, P3) for t in t_values])
            all_curves_points_t.append(curve_points)


        # Unir el último punto con el primero
        P0_last = torch.tensor([sewLine[0]['position']['x'], sewLine[0]['position']['y']])
        P3_first = torch.tensor([sewLine[-1]['position']['x'], sewLine[-1]['position']['y']])

        if 'control1' in sewLine[-1] and 'control2' in sewLine[-1]:
            P2_first = torch.tensor([sewLine[-1]['control1']['x'], sewLine[-1]['control1']['y']])
            P1_first = torch.tensor([sewLine[-1]['control2']['x'], sewLine[-1]['control2']['y']])
        else:
            P1_first = None
            P2_first = None

        # Verificamos si hay puntos de control y los incluimos en el cálculo
        if P1_first is not None and P2_first is not None:
            curve_points = torch.stack([bezier(t, P0_last, P1_first, P2_first, P3_first) for t in t_values])
        else:
            curve_points = torch.stack([bezier(t, P0_last, None, None, P3_first) for t in t_values])
        all_curves_points_t.append(curve_points)

    return torch.stack(all_curves_points_t)






def recalculate_boundary_vertex_tensor(curv, segment_index, t_value):
    # Obtener los puntos del segmento dado
    curve_points = curv[segment_index].clone().detach()

    # Calcular el índice correspondiente al valor de t
    index = int(t_value * (curve_points.size(0) - 1))
    
    # Devolver las coordenadas del punto en ese índice como un tensor
    return curve_points[index]

def compute_tensor_boundary_point(curv_points_tensor, boundary_vertex_segment_t_tensor):
    recalculated_points = []

    # Iterar sobre los puntos dentro del tensor
    for point_info in boundary_vertex_segment_t_tensor:
        # Obtener las coordenadas del punto, el índice del segmento y el valor de t
        x, y, segment_index, t_value = point_info.tolist()
        
        # Llamar a la función para recalcular el punto
        recalculated_point = recalculate_boundary_vertex_tensor(curv_points_tensor, int(segment_index), t_value)
        
        # Agregar el punto recalculado a la lista
        recalculated_points.append(recalculated_point)

    # Convertir la lista de puntos recalculados en un tensor
    recalculated_points_tensor = torch.stack(recalculated_points, dim=0)
    
    return recalculated_points_tensor


def fbezier(dif):
    pieces_bezierPath_tensor = copy.deepcopy(pieces_bezierPath)

    despX, despY = calcular_desplazamiento(pom_C0,pom_C1,poms[0]['orientation'],dif)
    if poms[0]['orientation']==0:
        # Desplazar los puntos de la sewline del POM
        sewLine_POM_0 = get_sewline_by_id(poms[0]['POM'][0][0]['piece'],pieces_bezierPath_tensor)
        desplazar_puntos_sewline_en_pom(sewLine_POM_0, poms[0]['POM'][0][0]['nodes']['sewLine'],despX,despY)
        update_sewline_in_pieces(sewLine_POM_0, poms[0]['POM'][0][0]['piece'],pieces_bezierPath_tensor)

        # Desplazar los puntos de la sewline del POM
        sewLine_POM_1 = get_sewline_by_id(poms[0]['POM'][1][0]['piece'],pieces_bezierPath_tensor)
        desplazar_puntos_sewline_en_pom(sewLine_POM_1, poms[0]['POM'][1][0]['nodes']['sewLine'],-despX,-despY)
        update_sewline_in_pieces(sewLine_POM_1, poms[0]['POM'][1][0]['piece'],pieces_bezierPath_tensor)
    else:
        sewLine_POM_1 = get_sewline_by_id(poms[0]['POM'][1][0]['piece'],pieces_bezierPath_tensor)
        desplazar_puntos_sewline_en_pom(sewLine_POM_1, poms[0]['POM'][1][0]['nodes']['sewLine'],despX,despY)
        update_sewline_in_pieces(sewLine_POM_1, poms[0]['POM'][1][0]['piece'],pieces_bezierPath_tensor)
       
    all_curve_points_tensor = bezier_curves_tensor(pieces_bezierPath_tensor)

    return all_curve_points_tensor


def fboundary(bezier_curvv_points_tensor):
    return compute_tensor_boundary_point(bezier_curvv_points_tensor,boundary_vertex_segment_t_tensor)


def fvertex(boundary_points_tensor):
    new_vertex_points = []
    for i, P in enumerate(points_array):
        recalculated_point_index = nearest_boundary_vertex_array_tensor[i].long().tolist()  # Convertir el tensor a una lista de Python
        recalculated_point = bilinear_interpolation(boundary_points_tensor[recalculated_point_index].tolist(), weights_array_tensor[i].tolist())
        new_vertex_points.append(torch.tensor(recalculated_point))
    return torch.stack(new_vertex_points)

original_pieces_bezierPath_tensor= fbezier(0)
original_boundary_points_tensor= fboundary(original_pieces_bezierPath_tensor)
original_vertex_points_tensor= fvertex(original_boundary_points_tensor)

original_edge_vertex_lengths = compute_edge_lengths(original_vertex_points_tensor,pieces_indices)

new_pieces_bezierPath_tensor= fbezier(0.7)
new_boundary_points_tensor= fboundary(new_pieces_bezierPath_tensor)
new_vertex_points_tensor= fvertex(new_boundary_points_tensor)
avatar1_edge_lengths = compute_edge_lengths(avatar1_3dpoints,pieces_indices)
avatar2_edge_lengths = compute_edge_lengths(avatar2_3dpoints,pieces_indices)
edge_vertex_lengths = compute_edge_lengths(new_vertex_points_tensor,pieces_indices)
diff_edges_2d = compute_edge_difference(edge_vertex_lengths, original_edge_vertex_lengths)
diff_edges_3d = compute_edge_difference(avatar2_edge_lengths, avatar1_edge_lengths)

indices = range(len(diff_edges_2d))

# Grafica las diferencias de bordes en función de los índices
plt.scatter(indices, diff_edges_2d, marker='o', color='blue')
plt.xlabel('Índices')
plt.ylabel('Diferencia de bordes')
plt.title('Diferencia de bordes por índice')
plt.grid(True)
plt.show()

indices = range(len(diff_edges_3d))

# Grafica las diferencias de bordes en función de los índices
plt.scatter(indices, diff_edges_3d, marker='o', color='blue')
plt.xlabel('Índices')
plt.ylabel('Diferencia de bordes')
plt.title('Diferencia de bordes por índice')
plt.grid(True)
plt.show()
target_strain = compute_total_strain(np.array(avatar2_edge_lengths),np.array(avatar1_edge_lengths)).flatten()
print("Diferencia en longitudes de aristas entre los dos avatares:")

# Crear el histograma
plt.hist(target_strain, bins=50, color='skyblue', edgecolor='black')

# Añadir etiquetas y título
plt.xlabel('Diferencia')
plt.ylabel('Frecuencia')
plt.title('STRAIN Histograma de % diferencias entre aristas de triángulos')

# Mostrar el histograma
plt.show()

edge_vertex_lengths_array = np.array(edge_vertex_lengths)
original_edge_vertex_lengths_array = np.array(original_edge_vertex_lengths)

# Calcula la diferencia entre los arrays
difference = np.abs(edge_vertex_lengths_array - original_edge_vertex_lengths_array)

y_index = np.arange(len(np.where(difference > 0)[0]))  # Índices para el eje y
x_index= difference[np.where(difference > 0)[0]]
# Grafica las diferencias en las coordenadas x
plt.figure(figsize=(8, 6))
plt.scatter(x_index, y_index, color='blue', label='Diferencias en X',s=1)
plt.xlabel('Diferencia')
plt.ylabel('Índice')
plt.title('Diferencias en X')
plt.legend()
plt.show()

# Imprime los índices y los valores donde hay cambios significativos
#threshold = 1.25 # Define un umbral para considerar un cambio significativo
#changed_indices = np.where(difference >threshold)[0]
#changed_values = difference[changed_indices]

#print(len(changed_indices))
#print("Índices de los cambios:", changed_indices)
#print("Valores de los cambios:", changed_values)

strain = np.abs(compute_total_strain(np.array(edge_vertex_lengths),np.array(original_edge_vertex_lengths)).flatten())
print("Diferencia en longitudes de aristas entre los dos avatares:")
threshold = 0.01 # Define un umbral para considerar un cambio significativo
changed_indices = np.where(strain >threshold)[0]
print(len(changed_indices))
# Crear el histograma
plt.hist(strain, bins=50, color='skyblue', edgecolor='black')

# Añadir etiquetas y título
plt.xlabel('Diferencia')
plt.ylabel('Frecuencia')
plt.title('STRAIN Histograma de % diferencias entre aristas de triángulos')

# Mostrar el histograma
plt.show()
# Visualizar los puntos de las curvas de Bézier
plt.figure(figsize=(8, 6))

for curve_points in new_pieces_bezierPath_tensor:
    curve_points = curve_points.numpy()  # Convertir tensor a array de NumPy
    plt.plot(curve_points[:, 0], curve_points[:, 1], color='blue', alpha=0.5)

plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.title('Curvas de Bézier')
plt.grid(True)
plt.show()


boundary_points__tensor_array = new_boundary_points_tensor.numpy()
# Extraer las coordenadas x e y
x = boundary_points__tensor_array[:, 0]
y = boundary_points__tensor_array[:, 1]

# Graficar los puntos
plt.scatter(x, y,s=1)
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.title('Puntos de límite')
plt.grid(True)
plt.show()

new_vertex_points__tensor_array = new_vertex_points_tensor.numpy()
# Extraer las coordenadas x e y
x = new_vertex_points__tensor_array[:, 0]
y = new_vertex_points__tensor_array[:, 1]

# Graficar los puntos
plt.scatter(x, y,s=1)
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.title('Puntos de límite')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np

# Supongamos que fbezier, fboundary, fvertex, compute_edge_lengths y compute_total_strain son funciones que ya tienes definidas
original_edge_length = compute_edge_lengths(original_vertex_points_tensor,pieces_indices)

# Función para calcular el costo
def cost_function2(optimized_pom, pieces_indices, original_edge_length, target_strain):
    new_pieces_bezierPath = fbezier(optimized_pom)
    new_boundary_points = fboundary(new_pieces_bezierPath)
    new_vertex_points = fvertex(new_boundary_points)
    #recalculated2_vertex_points_x = [point[0] for point in new_vertex_points]
    #recalculated2_vertex_points_y = [point[1] for point in new_vertex_points]

        # Dibuja los puntos originales y recalculados
    #plt.scatter(recalculated2_vertex_points_x, recalculated2_vertex_points_y, color='orange',s=1, label='Puntos recalculados')
    #plt.show()
    e_length = compute_edge_lengths(new_vertex_points, pieces_indices)
    new_strain = compute_total_strain(e_length, original_edge_length)
    #print(len(original_edge_length))
    strain_changed = new_strain[changed_indices]
    target_strain_changed = target_strain[changed_indices]
    #return np.mean((new_strain - target_strain) ** 2)
    return np.mean((strain_changed - target_strain_changed) ** 2)


def optimize_pom(initial_pom, pieces_indices, original_edge_length, target_strain, learning_rate, num_iterations):
    optimized_pom = initial_pom  # No necesitas hacer una copia si optimized_pom es un escalar
    losses = []

    for _ in range(num_iterations):
        loss = cost_function2(optimized_pom, pieces_indices, original_edge_length, target_strain)
        losses.append(loss)
        print(_,optimized_pom)
        # Calcula el gradiente utilizando la diferencia finita
        epsilon = 1e-6
        perturbation = epsilon
        loss_positive = cost_function2(optimized_pom + perturbation, pieces_indices, original_edge_length, target_strain)
        loss_negative = cost_function2(optimized_pom - perturbation, pieces_indices, original_edge_length, target_strain)
        grad = (loss_positive - loss_negative) / (2 * epsilon)

        # Actualiza optimized_pom utilizando el gradiente descendente
        optimized_pom -= learning_rate * grad

    return optimized_pom, losses

# Parámetros
initial_pom = 0  # Inicializa initial_pom como un escalar
learning_rate = 0.1
num_iterations =40

# Optimiza optimized_pom
optimized_pom, losses = optimize_pom(initial_pom, pieces_indices, original_edge_length, target_strain, learning_rate, num_iterations)
print("Optimized POM:", optimized_pom)

# Grafica el loss
plt.plot(range(num_iterations), losses)
plt.xlabel('Número de iteraciones')
plt.ylabel('Loss')
plt.title('Evolución del loss durante la optimización')
plt.show()

new_pieces_bezierPath = fbezier(optimized_pom)
new_boundary_points = fboundary(new_pieces_bezierPath)
new_vertex_points = fvertex(new_boundary_points)
recalculated2_vertex_points_x = [point[0] for point in new_vertex_points]
recalculated2_vertex_points_y = [point[1] for point in new_vertex_points]

        # Dibuja los puntos originales y recalculados
plt.scatter(recalculated2_vertex_points_x, recalculated2_vertex_points_y, color='orange',s=1, label='Puntos recalculados')
plt.show()


In [ ]:
print(losses)